## 1.Understanding Softmax

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable


# Cross entropy example
import numpy as np

In [3]:
# One hot
# 0: 1 0 0
# 1: 0 1 0
# 2: 0 0 1
Y = np.array([1, 0, 0])

Y_pred1 = np.array([0.7, 0.2, 0.1])
Y_pred2 = np.array([0.1, 0.3, 0.6])
print("loss1 = ", np.sum(-Y * np.log(Y_pred1)))
print("loss2 = ", np.sum(-Y * np.log(Y_pred2)))

# Softmax + CrossEntropy (logSoftmax + NLLLoss)
loss = nn.CrossEntropyLoss()

# target is of size nBatch
# each element in target has to have 0 <= value < nClasses (0-2)
# Input is class, not one-hot
Y = Variable(torch.LongTensor([0]), requires_grad=False)

# input is of size nBatch x nClasses = 1 x 4
# Y_pred are logits (not softmax)
Y_pred1 = Variable(torch.Tensor([[2.0, 1.0, 0.1]]))
Y_pred2 = Variable(torch.Tensor([[0.5, 2.0, 0.3]]))

l1 = loss(Y_pred1, Y)
l2 = loss(Y_pred2, Y)

print("PyTorch Loss1 = ", l1.data, "\nPyTorch Loss2=", l2.data)

print("Y_pred1=", torch.max(Y_pred1.data, 1)[1])
print("Y_pred2=", torch.max(Y_pred2.data, 1)[1])

# target is of size nBatch
# each element in target has to have 0 <= value < nClasses (0-2)
# Input is class, not one-hot
Y = Variable(torch.LongTensor([2, 0, 1]), requires_grad=False)

# input is of size nBatch x nClasses = 2 x 4
# Y_pred are logits (not softmax)
Y_pred1 = Variable(torch.Tensor([[0.1, 0.2, 0.9],
                                 [1.1, 0.1, 0.2],
                                 [0.2, 2.1, 0.1]]))


Y_pred2 = Variable(torch.Tensor([[0.8, 0.2, 0.3],
                                 [0.2, 0.3, 0.5],
                                 [0.2, 0.2, 0.5]]))

l1 = loss(Y_pred1, Y)
l2 = loss(Y_pred2, Y)

print("Batch Loss1 = ", l1.data, "\nBatch Loss2=", l2.data)

loss1 =  0.35667494393873245
loss2 =  2.3025850929940455
PyTorch Loss1 =  tensor(0.4170) 
PyTorch Loss2= tensor(1.8406)
Y_pred1= tensor([0])
Y_pred2= tensor([1])
Batch Loss1 =  tensor(0.4966) 
Batch Loss2= tensor(1.2389)


## Softmax for MNIST

In [4]:
train_dataset = datasets.MNIST(root='./data/mnist_data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./data/mnist_data/',
                              train=False,
                              transform=transforms.ToTensor())

Processing...
Done!


In [5]:
batch_size = 64
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)


In [6]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.l1 = nn.Linear(784, 520)
        self.l2 = nn.Linear(520, 320)
        self.l3 = nn.Linear(320, 240)
        self.l4 = nn.Linear(240, 120)
        self.l5 = nn.Linear(120, 10)

    def forward(self, x):
        x = x.view(-1, 784)  # Flatten the data (n, 1, 28, 28)-> (n, 784)
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return self.l5(x)



In [7]:
model = Net()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [11]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [13]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, target).item()
        # get the index of the max
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [14]:
for epoch in range(1, 10):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 1.418355
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.410044
Train Epoch: 1 [1280/60000 (2%)]	Loss: 1.155926
Train Epoch: 1 [1920/60000 (3%)]	Loss: 1.258758
Train Epoch: 1 [2560/60000 (4%)]	Loss: 1.247070
Train Epoch: 1 [3200/60000 (5%)]	Loss: 1.104152
Train Epoch: 1 [3840/60000 (6%)]	Loss: 1.044444
Train Epoch: 1 [4480/60000 (7%)]	Loss: 1.066652
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.909829
Train Epoch: 1 [5760/60000 (10%)]	Loss: 1.037709
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.998728
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.836295
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.888497
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.967065
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.898337
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.936628
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.820097
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.840762
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.622078
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.699638
Train Epoch: 1 [12800/60000 (

D:\Anaconda-installation\lib\site-packages\ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  



Test set: Average loss: 0.0065, Accuracy: 8783/10000 (87%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.435936
Train Epoch: 2 [640/60000 (1%)]	Loss: 0.330233
Train Epoch: 2 [1280/60000 (2%)]	Loss: 0.308586
Train Epoch: 2 [1920/60000 (3%)]	Loss: 0.335830
Train Epoch: 2 [2560/60000 (4%)]	Loss: 0.340799
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.379400
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.511329
Train Epoch: 2 [4480/60000 (7%)]	Loss: 0.482636
Train Epoch: 2 [5120/60000 (9%)]	Loss: 0.450332
Train Epoch: 2 [5760/60000 (10%)]	Loss: 0.289151
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.509003
Train Epoch: 2 [7040/60000 (12%)]	Loss: 0.349608
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.543365
Train Epoch: 2 [8320/60000 (14%)]	Loss: 0.264706
Train Epoch: 2 [8960/60000 (15%)]	Loss: 0.199028
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.339305
Train Epoch: 2 [10240/60000 (17%)]	Loss: 0.531002
Train Epoch: 2 [10880/60000 (18%)]	Loss: 0.417482
Train Epoch: 2 [11520/60000 (19%)]	Loss: 0.295224
Train Epoch: 2 [12

Train Epoch: 3 [43520/60000 (72%)]	Loss: 0.176579
Train Epoch: 3 [44160/60000 (74%)]	Loss: 0.319166
Train Epoch: 3 [44800/60000 (75%)]	Loss: 0.174943
Train Epoch: 3 [45440/60000 (76%)]	Loss: 0.388972
Train Epoch: 3 [46080/60000 (77%)]	Loss: 0.119787
Train Epoch: 3 [46720/60000 (78%)]	Loss: 0.252668
Train Epoch: 3 [47360/60000 (79%)]	Loss: 0.096680
Train Epoch: 3 [48000/60000 (80%)]	Loss: 0.137992
Train Epoch: 3 [48640/60000 (81%)]	Loss: 0.246826
Train Epoch: 3 [49280/60000 (82%)]	Loss: 0.455590
Train Epoch: 3 [49920/60000 (83%)]	Loss: 0.334881
Train Epoch: 3 [50560/60000 (84%)]	Loss: 0.260753
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.255523
Train Epoch: 3 [51840/60000 (86%)]	Loss: 0.150571
Train Epoch: 3 [52480/60000 (87%)]	Loss: 0.241700
Train Epoch: 3 [53120/60000 (88%)]	Loss: 0.168153
Train Epoch: 3 [53760/60000 (90%)]	Loss: 0.352058
Train Epoch: 3 [54400/60000 (91%)]	Loss: 0.110570
Train Epoch: 3 [55040/60000 (92%)]	Loss: 0.104747
Train Epoch: 3 [55680/60000 (93%)]	Loss: 0.350753


Train Epoch: 5 [26880/60000 (45%)]	Loss: 0.095516
Train Epoch: 5 [27520/60000 (46%)]	Loss: 0.228894
Train Epoch: 5 [28160/60000 (47%)]	Loss: 0.207344
Train Epoch: 5 [28800/60000 (48%)]	Loss: 0.291479
Train Epoch: 5 [29440/60000 (49%)]	Loss: 0.144002
Train Epoch: 5 [30080/60000 (50%)]	Loss: 0.040947
Train Epoch: 5 [30720/60000 (51%)]	Loss: 0.064059
Train Epoch: 5 [31360/60000 (52%)]	Loss: 0.217987
Train Epoch: 5 [32000/60000 (53%)]	Loss: 0.141434
Train Epoch: 5 [32640/60000 (54%)]	Loss: 0.222761
Train Epoch: 5 [33280/60000 (55%)]	Loss: 0.050973
Train Epoch: 5 [33920/60000 (57%)]	Loss: 0.286679
Train Epoch: 5 [34560/60000 (58%)]	Loss: 0.244287
Train Epoch: 5 [35200/60000 (59%)]	Loss: 0.178173
Train Epoch: 5 [35840/60000 (60%)]	Loss: 0.206312
Train Epoch: 5 [36480/60000 (61%)]	Loss: 0.177187
Train Epoch: 5 [37120/60000 (62%)]	Loss: 0.083425
Train Epoch: 5 [37760/60000 (63%)]	Loss: 0.208984
Train Epoch: 5 [38400/60000 (64%)]	Loss: 0.259833
Train Epoch: 5 [39040/60000 (65%)]	Loss: 0.202019


Train Epoch: 7 [10240/60000 (17%)]	Loss: 0.056456
Train Epoch: 7 [10880/60000 (18%)]	Loss: 0.057161
Train Epoch: 7 [11520/60000 (19%)]	Loss: 0.039307
Train Epoch: 7 [12160/60000 (20%)]	Loss: 0.023793
Train Epoch: 7 [12800/60000 (21%)]	Loss: 0.133913
Train Epoch: 7 [13440/60000 (22%)]	Loss: 0.041901
Train Epoch: 7 [14080/60000 (23%)]	Loss: 0.054671
Train Epoch: 7 [14720/60000 (25%)]	Loss: 0.088813
Train Epoch: 7 [15360/60000 (26%)]	Loss: 0.024570
Train Epoch: 7 [16000/60000 (27%)]	Loss: 0.082379
Train Epoch: 7 [16640/60000 (28%)]	Loss: 0.118667
Train Epoch: 7 [17280/60000 (29%)]	Loss: 0.025334
Train Epoch: 7 [17920/60000 (30%)]	Loss: 0.077692
Train Epoch: 7 [18560/60000 (31%)]	Loss: 0.019143
Train Epoch: 7 [19200/60000 (32%)]	Loss: 0.057741
Train Epoch: 7 [19840/60000 (33%)]	Loss: 0.170061
Train Epoch: 7 [20480/60000 (34%)]	Loss: 0.156805
Train Epoch: 7 [21120/60000 (35%)]	Loss: 0.122314
Train Epoch: 7 [21760/60000 (36%)]	Loss: 0.226396
Train Epoch: 7 [22400/60000 (37%)]	Loss: 0.069447


Train Epoch: 8 [54400/60000 (91%)]	Loss: 0.034298
Train Epoch: 8 [55040/60000 (92%)]	Loss: 0.120902
Train Epoch: 8 [55680/60000 (93%)]	Loss: 0.052397
Train Epoch: 8 [56320/60000 (94%)]	Loss: 0.046905
Train Epoch: 8 [56960/60000 (95%)]	Loss: 0.074477
Train Epoch: 8 [57600/60000 (96%)]	Loss: 0.037963
Train Epoch: 8 [58240/60000 (97%)]	Loss: 0.129280
Train Epoch: 8 [58880/60000 (98%)]	Loss: 0.148916
Train Epoch: 8 [59520/60000 (99%)]	Loss: 0.059969

Test set: Average loss: 0.0017, Accuracy: 9682/10000 (96%)

Train Epoch: 9 [0/60000 (0%)]	Loss: 0.030786
Train Epoch: 9 [640/60000 (1%)]	Loss: 0.065120
Train Epoch: 9 [1280/60000 (2%)]	Loss: 0.013893
Train Epoch: 9 [1920/60000 (3%)]	Loss: 0.025464
Train Epoch: 9 [2560/60000 (4%)]	Loss: 0.020509
Train Epoch: 9 [3200/60000 (5%)]	Loss: 0.028029
Train Epoch: 9 [3840/60000 (6%)]	Loss: 0.058152
Train Epoch: 9 [4480/60000 (7%)]	Loss: 0.015301
Train Epoch: 9 [5120/60000 (9%)]	Loss: 0.063291
Train Epoch: 9 [5760/60000 (10%)]	Loss: 0.331330
Train Epoch: